In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl

import scipy.sparse as sp

mpl.rcParams["figure.dpi"] = 150

In [2]:
# Initialize saturation map
LONG_LEG_LENGTH = 7 # must be odd
INITIAL_HOMO_SAT = 1

sat_array_empty = np.zeros((LONG_LEG_LENGTH, (LONG_LEG_LENGTH+1)//2))


In [3]:
# Function blocks

def construct_simulation_cell_mask(sat_arr):
    """Constructs mask of all cells that are active in the simulation
    """
    simulation_cell_mask = np.copy(sat_arr)
    
    for i in range(LONG_LEG_LENGTH):
        width = (i+2)//2
        simulation_cell_mask[LONG_LEG_LENGTH - i-1,:width] = np.full(width, 1)

    return simulation_cell_mask

# def construct_diffusion_mask(sim_cell_mask)



In [4]:
# Default seed 


In [5]:
A = np.array(
    [
        [
            np.eye(2), np.eye(2)
        ],
        [
            np.eye(2), np.eye(2)
        ]
        ]
)

In [6]:
A[:,:,:,:]

array([[[[1., 0.],
         [0., 1.]],

        [[1., 0.],
         [0., 1.]]],


       [[[1., 0.],
         [0., 1.]],

        [[1., 0.],
         [0., 1.]]]])

In [7]:
A[0,0,:,:]

array([[1., 0.],
       [0., 1.]])

In [8]:
Arr = np.empty((10,10,10,10))

for i in range(10):
    for j in range(10):
        temp_array = np.zeros((10,10))
        temp_array[i,j] = 2

        Arr[i,j,:,:] = temp_array

In [9]:
Arr[0,1,:,:]

array([[0., 2., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [10]:
diffusor_submask_inner = np.array(
    [
        [0, 1/6, 1/6],
        [1/6, 0, 1/6],
        [1/6, 1/6, 0]
    ]
)

diffusor_submask_lower_ragged = np.array(
    [
        [0, 1/3],
        [1/3, 0],
        [1/3, 0]
    ]
)

diffusor_submask_upper_ragged = np.array(
    [
        [0, 1/6, 1/6],
        [1/3, 0, 0],
        [1/6, 1/6, 0]
    ]
)

diffusor_submask_flush = np.array(
    [
        [1/6, 1/3],
        [0, 1/3],
        [1/6, 0]
    ]
)

In [11]:
def sum_NN(a, indices):
    """Sums values of nearest neighbors in a hexagonal lattice
    
    """
    # if index is legit, append
    # if index is SOMEVALUE, add to list
    # create mask for tensor

### TO-DO

- Make diffusor into a class


In [12]:
class Converter:
    """
    A class that allows for conversion from cartesian coordinates
    to index in flattened list and vice-versa for custom hexagon-based
    simulation area.

    Attributes:
        L (int): The vertical length of simulation grid, always odd.

    Methods: FIIILLLLL OUUUTTTT

    """
    def __init__(self, L):
        self.L = L 
        self.W = int((self.L+1)/2) # Calculates horizontal length of grid
        self.total_indices = int(self.W*self.W) # Uses Gauss' formula to calculate total size (see doc)

        ##### construct list of coords #####

        cartesian_vertical_coord_list = np.ceil(2*np.sqrt(np.arange(0,self.total_indices)+1) - 1)
        cartesian_horizontal_coord_list = np.floor((cartesian_vertical_coord_list+1)/2) - np.floor(np.square((cartesian_vertical_coord_list+1)/2))

        self.coord_array = np.empty((self.total_indices, 2), dtype=int)

        for i in range(self.total_indices):
            self.coord_array[i, :] = (self.L - cartesian_vertical_coord_list[i], cartesian_horizontal_coord_list[i] + i)

        ##### construct array of indices #####

        self.index_array = np.full((self.L, self.W), np.nan)
        for i in range(self.total_indices):
            self.index_array[self.coord_array[i,0], self.coord_array[i,1]] = i
        
    def __repr__(self):
        """
        Reserved __repr__ method.

        Arguments:
            None

        Returns:
            (str): Displays total vertical and horizontal breadth of array considered.
        """
        return f"L = {self.L}; W = {self.W}"

    def coords_to_index(self, coords):
        """
        A method that converts array coordinates of a site to 
        its position in flattened array.

        Arguments:
            coords (1D array): array coordinates of a simulation site
        
        Returns:
            (int) Index in the 1D flattened representation.
        """
        return int(self.index_array[coords[0], coords[1]])

    def index_to_coords(self, index:int):
        """
        A method that converts position in flattened array to 
        its coordinates in two dimensionnal form.

        Arguments:
            index (int): index to be converted to coordinates
        
        Returns:
            (int): line of the simulation site
            (int): column of the simulation site
        """
        coords = self.coord_array[index]
        return coords[0], coords[1]

    def submask_flattener(self, submask, location_coords, center_coords):
        """
        FKLDJALKFJLKDASJFKLADS
        """
        submask_shape = np.shape(submask)
        flattened_array = np.zeros(self.total_indices)
    
        for line in range(submask_shape[0]):
            for col in range(submask_shape[1]):
                
                submask_elem = submask[line, col]
                physical_line = line - center_coords[0] + location_coords[0]
                physical_col = col - center_coords[1] + location_coords[1]

                flattened_array[self.coords_to_index((physical_line, physical_col))] = submask_elem

        return flattened_array



In [31]:
class Diffusor:
    """
    A class that manages the diffusion of the water 
    saturation field and its boundary conditions near 
    ice cells through a relaxation style method.

    Attributes:
        L (int): Vertical breadth of simulation sites
        converter_obj (obj): Pre-initialized instance of the `Converter` class

    Methods:
        À FAIRE LOLOLOLOLOLOL
    """

    diffusor_submask_inner = np.array(
        [
            [0, 1/6, 1/6],
            [1/6, 0, 1/6],
            [1/6, 1/6, 0]
        ]
    )

    diffusor_submask_lower_ragged = np.array(
        [
            [0, 1/3],
            [1/3, 0],
            [1/3, 0]
        ]
    )

    diffusor_submask_upper_ragged = np.array(
        [
            [0, 1/6, 1/6],
            [1/3, 0, 0],
            [1/6, 1/6, 0]
        ]
    )

    diffusor_submask_flush = np.array(
        [
        [1/6, 1/3],
        [0, 1/3],
        [1/6, 0]
        ]
    )

    def __init__(self, L, converter_obj):
        """
        AFIREA DFJASDFLKADJFLAKJDAKFJADSKLFJASDKLFJAS
        """

        self.L = int(L)
        self.W = int((L+1)/2)
        self.total_indices = int(self.W*self.W) # Uses Gauss' formula to calculate total size (see doc)
        self.converter_obj = converter_obj


        # for line in range(self.L):
        #     for col in range(self.W):
        #         converter_obj.coords_to_index(1)

    def construct_default_diffusion_matrix(self):

        diffusion_matrix = sp.lil_matrix((self.total_indices, self.total_indices)) # initialize diffusion matrix

        ##### Set top row to zero diffusion #####
        for col in range(self.W):
            index = self.converter_obj.coords_to_index((0, col))
            diffusion_matrix[index, index] = 1 # sets up diffusion matrix such that the top row isn't subject to diffusion

        ##### Set leftmost cells to diffuse following reflexion boundary conditions #####
        for line in range(1, self.L-3):
            index = self.converter_obj.coords_to_index((line, 0))

            # diffusion_matrix[index, :] = self.converter_obj.submask_flattener(self.diffusor_submask_flush, (line, 0), (1,0))
            print(self.converter_obj.submask_flattener(self.diffusor_submask_flush, (line, 0), (1,0)))

        ##### Set upper ragged matrix elements #####
        for col in range(3, self.W-1):
            line = self.L - 2*(col+1)
            index = self.converter_obj.coords_to_index((line, col))

            diffusion_matrix[index, :] = self.converter_obj.submask_flattener(self.diffusor_submask_upper_ragged, (line, col), (1,1))

        ##### Set lower ragged matrix elements #####
        for col in range(2, self.W-1):
            line = self.L - 2*col - 1
            index = self.converter_obj.coords_to_index((line, col))

            diffusion_matrix[index, :] = self.converter_obj.submask_flattener(self.diffusor_submask_lower_ragged, (line, col), (1,1))
            
        ##### Set non-edge-case matrix elements #####
        for col in range(1, self.W-2):
            for line in range(1, self.L-2*col-3):
                index = self.converter_obj.coords_to_index((line, col))

                diffusion_matrix[index, :] = self.converter_obj.submask_flattener(self.diffusor_submask_inner, (line, col), (1,1))
        


In [32]:
gaming = Converter(1001)
gamig = Diffusor(10, gaming)

In [33]:
diffusor_submask_inner = np.array(
        [
            [0, 1/6, 1/6],
            [1/6, 0, 1/6],
            [1/6, 1/6, 0]
        ]
    )
Array = gaming.submask_flattener(diffusor_submask_inner, [1,1], [1,1])

In [34]:
A = np.empty((1000,1000), dtype="bool")
B = np.empty((1000,1000), dtype="float")

In [35]:
from sys import getsizeof
print(getsizeof(A))
print(getsizeof(B))

1000128
8000128


In [36]:
conv = Converter(1001)
gaming = Diffusor(1001, conv)
gaming.construct_default_diffusion_matrix()

[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0.

ValueError: cannot convert float NaN to integer

In [ ]:
from scipy.sparse import csr_matrix

A = csr_matrix((10,10))

In [ ]:
A[0,1]

0.0

In [39]:
d = dict()
d[(1,1)] = 2

In [43]:
import timeit

timeit.timeit("d[(1,1)]", setup="d=dict(); d[(1,1)]=2")

0.043846002999998746